In [1]:
from tqdm import tqdm
import time
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import skimage.io as io
from skimage.io import imread_collection
from sklearn import preprocessing
from skimage.feature import hog
import matplotlib.pyplot as plt
import os
import glob
from PIL import Image, ImageFilter

In [2]:
from sklearn import linear_model,svm,metrics
from sklearn.model_selection import train_test_split

In [3]:
X = pd.read_csv('train.csv')
Y = pd.read_csv('test.csv')

In [6]:
xtrain = X.iloc[:,2:902]
ytrain = X['900']
xtest = Y.iloc[:,1:901]
ytest = Y['900']

In [8]:
print(xtrain,ytrain)
print(xtrain.shape,ytrain.shape)

              0         1         2         3         4         5         6  \
0      0.236927  0.269290  0.260817  0.024606  0.187094  0.013559  0.030569   
1      0.375347  0.256646  0.018823  0.013986  0.139792  0.000000  0.003092   
2      0.301683  0.301683  0.021224  0.001865  0.008142  0.000000  0.001279   
3      0.113634  0.375097  0.166437  0.038052  0.100244  0.017011  0.060997   
4      0.012661  0.008612  0.005314  0.003734  0.011690  0.002801  0.010183   
...         ...       ...       ...       ...       ...       ...       ...   
28704  0.095519  0.125643  0.023657  0.036335  0.064321  0.015050  0.027599   
28705  0.288954  0.246246  0.028479  0.022261  0.094003  0.023888  0.017042   
28706  0.248773  0.338765  0.000000  0.148120  0.243281  0.014438  0.030707   
28707  0.297100  0.295038  0.056008  0.096251  0.026158  0.010851  0.019044   
28708  0.162404  0.259857  0.259857  0.041571  0.176210  0.005086  0.022469   

              7         8         9  ...       890 

In [9]:
print(xtest,ytest)
print(xtest.shape,ytest.shape)

             0         1         2         3         4         5         6  \
0     0.256441  0.153981  0.256441  0.100254  0.152125  0.024297  0.182082   
1     0.095931  0.054529  0.198330  0.186632  0.339750  0.239490  0.074035   
2     0.289277  0.051690  0.000000  0.011619  0.085676  0.165210  0.074938   
3     0.125531  0.026772  0.023381  0.010722  0.289026  0.126535  0.219998   
4     0.007818  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
7173  0.257689  0.210449  0.001226  0.024208  0.010400  0.020849  0.038666   
7174  0.232570  0.096617  0.108412  0.063117  0.084375  0.041499  0.017555   
7175  0.225891  0.224982  0.244414  0.145955  0.177314  0.031713  0.000000   
7176  0.290448  0.290448  0.068049  0.021182  0.075819  0.008791  0.000000   
7177  0.242859  0.240453  0.048992  0.030137  0.120691  0.024080  0.026951   

             7         8         9  ...       890       891    

In [18]:
clf = svm.SVC(kernel='rbf', degree=1)
clf.fit(xtrain,ytrain)
yhat = clf.predict(xtest)
print(metrics.accuracy_score(ytest,yhat))

0.5321816662022848


In [57]:
from sklearn.neighbors import KNeighborsClassifier
K = 7
neigh = KNeighborsClassifier(n_neighbors=K).fit(xtrain,ytrain)
yhatKNN = neigh.predict(xtest)
print(metrics.accuracy_score(ytest,yhatKNN))

0.46447478406241294


In [68]:
tree = DecisionTreeClassifier(max_depth=3,random_state=42)
tree.fit(xtrain,ytrain)
yhattree = tree.predict(xtest)
print(metrics.accuracy_score(ytest,yhattree))

0.28893842295904154


In [73]:
for max_d in range(1,21):
    model = DecisionTreeClassifier(max_depth=max_d, random_state=42)
    model.fit(xtrain,ytrain)
    print('The Training Accuracy for max_depth {} is:'.format(max_d), model.score(xtrain,ytrain))
    print('The Validation Accuracy for max_depth {} is:'.format(max_d), model.score(xtest,ytest))
    print('')

The Training Accuracy for max_depth 1 is: 0.2513149186666202
The Validation Accuracy for max_depth 1 is: 0.24714405126776262

The Training Accuracy for max_depth 2 is: 0.28848096415758123
The Validation Accuracy for max_depth 2 is: 0.2832265254945667

The Training Accuracy for max_depth 3 is: 0.29502943327876274
The Validation Accuracy for max_depth 3 is: 0.28893842295904154

The Training Accuracy for max_depth 4 is: 0.3187850499843255
The Validation Accuracy for max_depth 4 is: 0.3138757314015046

The Training Accuracy for max_depth 5 is: 0.3336236023546623
The Validation Accuracy for max_depth 5 is: 0.3200055725828922

The Training Accuracy for max_depth 6 is: 0.3504475948308893
The Validation Accuracy for max_depth 6 is: 0.3225132348843689

The Training Accuracy for max_depth 7 is: 0.37427287610157095
The Validation Accuracy for max_depth 7 is: 0.3258567846196712

The Training Accuracy for max_depth 8 is: 0.40711971855515694
The Validation Accuracy for max_depth 8 is: 0.323209807745

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
LR = LogisticRegression(C=0.01,solver='liblinear').fit(xtrain,ytrain)
yhatLR = LR.predict(xtest)
print(LR.predict_proba(xtest))

[[0.13883522 0.01246783 0.09510223 ... 0.30553867 0.29942576 0.0493979 ]
 [0.20288571 0.01755619 0.14295456 ... 0.1347073  0.1919263  0.01745911]
 [0.20320426 0.01211616 0.15967852 ... 0.14070932 0.2820327  0.07611405]
 ...
 [0.05322208 0.00967311 0.12429341 ... 0.12849242 0.07007251 0.48300722]
 [0.25874617 0.04068548 0.22618293 ... 0.07160503 0.08975757 0.26066724]
 [0.10299196 0.00978886 0.16904958 ... 0.17253547 0.08282248 0.30231242]]


In [48]:
print(confusion_matrix(ytest,yhatLR))
print(classification_report(ytest,yhatLR))

[[ 189    0   62  310  181  166   50]
 [  34    0    8   36    9   20    4]
 [  86    0  158  322  187  152  119]
 [  46    0   39 1472   78  117   22]
 [  63    0   65  358  541  151   55]
 [ 122    0   72  407  275  333   38]
 [  47    0   60  174  101   57  392]]
              precision    recall  f1-score   support

       angry       0.32      0.20      0.24       958
     disgust       0.00      0.00      0.00       111
        fear       0.34      0.15      0.21      1024
       happy       0.48      0.83      0.61      1774
     neutral       0.39      0.44      0.42      1233
         sad       0.33      0.27      0.30      1247
    surprise       0.58      0.47      0.52       831

    accuracy                           0.43      7178
   macro avg       0.35      0.34      0.33      7178
weighted avg       0.40      0.43      0.40      7178



C:\Users\AnirudhCV\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AnirudhCV\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AnirudhCV\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
from sklearn.metrics import f1_score
print(f1_score(ytest,yhatLR,average='weighted'))

0.3958746528798058
